In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import matplotlib as mpl

# 设置字体以支持中文显示
mpl.rcParams['font.family'] = 'sans-serif'
mpl.rcParams['font.sans-serif'] = ['SimHei']
mpl.rcParams['axes.unicode_minus'] = False

# 读取数据
file_path = ''  # 替换为你的数据文件路径
data = pd.read_excel(file_path)

# 将数据转换为数值类型
data.iloc[:, 1:12] = data.iloc[:, 1:12].apply(pd.to_numeric, errors='coerce')

# 处理缺失值
data.dropna(inplace=True)

# 选择特征和目标变量
X = data.iloc[:, 1:9].values  # 选择第二列到第九列作为输入特征
y = data.iloc[:, 9:11].values  # 选择第十和第十一列作为输出目标（裂缝倾角和裂缝宽度）

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 数据标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 构建深度神经网络模型
model = keras.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)))
model.add(layers.Dropout(0.2))  # 添加Dropout层以防止过拟合
model.add(layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)))
model.add(layers.Dense(2))  # 输出2个参数：裂缝倾角、裂缝宽度

# 使用Adam优化器，设置学习率
optimizer = Adam(learning_rate=0.001)

# 编译模型
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

# 训练模型
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2)

# 评估模型
loss, mae = model.evaluate(X_test, y_test)
print(f"测试集均方误差: {loss:.4f}")
print(f"测试集平均绝对误差: {mae:.4f}")

# 预测
predictions = model.predict(X_test)

# 计算裂缝倾角和裂缝宽度的 R² 值
r2_values = {
    "裂缝倾角": {
        "训练集": r2_score(y_train[:, 0], model.predict(X_train)[:, 0]),
        "测试集": r2_score(y_test[:, 0], predictions[:, 0])
    },
    "裂缝宽度": {
        "训练集": r2_score(y_train[:, 1], model.predict(X_train)[:, 1]),
        "测试集": r2_score(y_test[:, 1], predictions[:, 1])
    }
}

# 输出裂缝倾角和裂缝宽度的 R² 值
for feature, r2 in r2_values.items():
    print(f"{feature} 训练集 R²: {r2['训练集']:.4f}, 测试集 R²: {r2['测试集']:.4f}")

# 计算裂缝密度（简单加权组合：裂缝宽度与倾角的比值，作为示例）
fracture_density = predictions[:, 1] / (np.cos(np.radians(predictions[:, 0])) + 1e-6)  # 避免除以零

# 将结果保存到Excel
results_df = pd.DataFrame(predictions, columns=['预测裂缝倾角', '预测裂缝宽度'])
results_df['裂缝密度'] = fracture_density  # 添加裂缝密度列
results_df.to_excel('predicted_fracture_features_with_density.xlsx', index=False)

print("预测结果已保存至predicted_fracture_features_with_density.xlsx")


KeyboardInterrupt: 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score

# 设置中文、字体、负号正常显示
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体
plt.rcParams['font.family'] = 'Times New Roman'  # 英文字体
plt.rcParams['axes.unicode_minus'] = False  # 负号正常显示

# 字体大小设置
title_fontsize = 20
label_fontsize = 18
tick_fontsize = 16
annot_fontsize = 16
legend_fontsize = 16

# 创建玫瑰图画布（提高dpi）
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8), subplot_kw=dict(polar=True), dpi=150)

# 角度转弧度
actual_angles_rad = np.deg2rad(y_test[:, 0])
pred_angles_rad = np.deg2rad(predictions[:, 0])
bins = 16

# --- 玫瑰图1：实际裂缝倾角 ---
ax1.set_title('Actual crack dip rose diagram', fontsize=title_fontsize, color='black')
n, bins_actual = np.histogram(actual_angles_rad, bins=bins, density=True)
bin_centers_actual = 0.5 * (bins_actual[1:] + bins_actual[:-1])
ax1.bar(bin_centers_actual, n, width=np.pi/bins, color='blue', alpha=0.7, edgecolor='w')
ax1.tick_params(labelsize=tick_fontsize)

for angle, height in zip(bin_centers_actual, n):
    ax1.plot([angle, angle], [0, height], color='black', linewidth=1.5)
    if height > 1:
        ax1.plot([angle - np.pi/bins/2, angle + np.pi/bins/2], [height, height], color='black', linewidth=1.5)
        ax1.plot([angle - np.pi/bins/2, angle - np.pi/bins/2], [0, height], color='black', linewidth=1.5)
        ax1.plot([angle + np.pi/bins/2, angle + np.pi/bins/2], [0, height], color='black', linewidth=1.5)
        ax1.plot([angle - np.pi/bins/2, angle + np.pi/bins/2], [0, 0], color='black', linewidth=1.5)

# --- 玫瑰图2：预测裂缝倾角 ---
ax2.set_title('Predicted crack dip rose diagram', fontsize=title_fontsize, color='black')
n_pred, bins_pred = np.histogram(pred_angles_rad, bins=bins, density=True)
bin_centers_pred = 0.5 * (bins_pred[1:] + bins_pred[:-1])
ax2.bar(bin_centers_pred, n_pred, width=np.pi/bins, color='green', alpha=0.7, edgecolor='w')
ax2.tick_params(labelsize=tick_fontsize)

for angle, height in zip(bin_centers_pred, n_pred):
    ax2.plot([angle, angle], [0, height], color='black', linewidth=1.5)
    if height > 1:
        ax2.plot([angle - np.pi/bins/2, angle + np.pi/bins/2], [height, height], color='black', linewidth=1.5)
        ax2.plot([angle - np.pi/bins/2, angle - np.pi/bins/2], [0, height], color='black', linewidth=1.5)
        ax2.plot([angle + np.pi/bins/2, angle + np.pi/bins/2], [0, height], color='black', linewidth=1.5)
        ax2.plot([angle - np.pi/bins/2, angle + np.pi/bins/2], [0, 0], color='black', linewidth=1.5)

# 创建拟合散点图画布（提高dpi）
fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(18, 8), dpi=150)

# 模型预测
train_pred = model.predict(X_train)[:, 0]
test_pred = predictions[:, 0]

# 计算拟合精度和R²
train_accuracy = 100 * (1 - np.mean(np.abs((y_train[:, 0] - train_pred) / y_train[:, 0])))
train_r2 = r2_score(y_train[:, 0], train_pred)

test_accuracy = 100 * (1 - np.mean(np.abs((y_test[:, 0] - test_pred) / y_test[:, 0])))
test_r2 = r2_score(y_test[:, 0], test_pred)

# --- 散点图1：训练集 ---
ax3.scatter(y_train[:, 0], train_pred, c='blue', s=100, alpha=0.6,
            edgecolor='black', label='Training set')
ax3.plot([0, 90], [0, 90], color='red', linestyle='--', label='Ideal fit line')
ax3.set_title('Training set actual crack inclination vs Predicting crack inclination', fontsize=title_fontsize)
ax3.set_xlabel('Actual crack inclination (°)', fontsize=label_fontsize)
ax3.set_ylabel('Predicting crack inclination (°)', fontsize=label_fontsize)
ax3.legend(fontsize=legend_fontsize)
ax3.grid(False)
ax3.tick_params(direction='in', labelsize=tick_fontsize)
# 训练集标签位置下移
ax3.annotate(f'Fitting accuracy: {train_accuracy:.2f}%', xy=(0.05, 0.85), xycoords='axes fraction',
             fontsize=annot_fontsize, color='black', bbox=dict(facecolor='white', alpha=0.6))


# --- 散点图2：测试集 ---
ax4.scatter(y_test[:, 0], test_pred, c='green', s=100, alpha=0.6,
            edgecolor='black', label='Test set')
ax4.plot([0, 90], [0, 90], color='red', linestyle='--', label='Ideal fit line')
ax4.set_title('Test set actual crack inclination vs Predicting crack inclination', fontsize=title_fontsize)
ax4.set_xlabel('Actual crack inclination (°)', fontsize=label_fontsize)
ax4.set_ylabel('Predicting crack inclination (°)', fontsize=label_fontsize)
ax4.legend(fontsize=legend_fontsize)
ax4.grid(False)
ax4.tick_params(direction='in', labelsize=tick_fontsize)
# 测试集标签位置下移
ax4.annotate(f'Fitting accuracy: {test_accuracy:.2f}%', xy=(0.05, 0.85), xycoords='axes fraction',
             fontsize=annot_fontsize, color='black', bbox=dict(facecolor='white', alpha=0.6))


# 自动调整子图间距
plt.tight_layout()

# 可选保存图像
# fig.savefig("rose_diagrams.png", dpi=300)
# fig2.savefig("scatter_fitting.png", dpi=300)

# 显示图形
plt.show()



KeyboardInterrupt: 

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import r2_score

plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体
plt.rcParams['axes.unicode_minus'] = False    # 正确显示负号

# 设置字号变量
title_fontsize = 20
label_fontsize = 16
legend_fontsize = 16
annot_fontsize = 16

# 创建画布和子图（直方图）
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8), dpi=150)

# 1. 实际裂缝宽度的直方图
ax1.set_title('Histogram of actual crack width', fontsize=title_fontsize)
sns.histplot(y_test[:, 1], bins=30, color='blue', kde=True, ax=ax1, stat='count', label='Actual crack width', fill=True)
ax1.set_xlabel('Crack width (cm)', fontsize=label_fontsize)
ax1.set_ylabel('Frequency', fontsize=label_fontsize)
ax1.legend(fontsize=legend_fontsize)
ax1.grid(False)

# 2. 预测裂缝宽度的直方图
ax2.set_title('Histogram of predicted crack width', fontsize=title_fontsize)
sns.histplot(predictions[:, 1], bins=30, color='orange', kde=True, ax=ax2, stat='count', label='Predicted crack width', fill=True)
ax2.set_xlabel('Crack width (cm)', fontsize=label_fontsize)
ax2.set_ylabel('Frequency', fontsize=label_fontsize)
ax2.legend(fontsize=legend_fontsize)
ax2.grid(False)

# 创建画布和子图（拟合散点图）
fig2, (ax3, ax4) = plt.subplots(1, 2, figsize=(18, 8))

# 计算拟合精度
train_pred = model.predict(X_train)[:, 1]
test_pred = predictions[:, 1]

train_accuracy = 100 * (1 - np.mean(np.abs((y_train[:, 1] - train_pred) / y_train[:, 1])))
test_accuracy = 100 * (1 - np.mean(np.abs((y_test[:, 1] - test_pred) / y_test[:, 1])))

# 3. 训练集拟合图
ax3.scatter(
    y_train[:, 1],
    train_pred,
    c='green',
    s=100,
    alpha=0.7,
    edgecolor='black',
    label='Training set'
)
max_val_train = max(y_train[:, 1].max(), train_pred.max())
ax3.plot([0, max_val_train], [0, max_val_train], color='red', linestyle='--', label='Ideal fit line')  # 45度参考线
ax3.set_title('Training set actual crack width vs Predicted crack width', fontsize=title_fontsize)
ax3.set_xlabel('Actual crack width (cm)', fontsize=label_fontsize)
ax3.set_ylabel('Predicted crack width (cm)', fontsize=label_fontsize)
ax3.legend(fontsize=legend_fontsize)
ax3.grid(False)
ax3.annotate(f'Fitting accuracy: {train_accuracy:.2f}%', xy=(0.05, 0.85), xycoords='axes fraction',
             fontsize=annot_fontsize, color='black', bbox=dict(facecolor='white', alpha=0.6))

# 4. 测试集拟合图
ax4.scatter(
    y_test[:, 1],
    test_pred,
    c='orange',
    s=100,
    alpha=0.7,
    edgecolor='black',
    label='Test set'
)
max_val_test = max(y_test[:, 1].max(), test_pred.max())
ax4.plot([0, max_val_test], [0, max_val_test], color='red', linestyle='--', label='Ideal fit line')  # 45度参考线
ax4.set_title('Test set actual crack width vs Predicted crack width', fontsize=title_fontsize)
ax4.set_xlabel('Actual crack width (cm)', fontsize=label_fontsize)
ax4.set_ylabel('Predicted crack width (cm)', fontsize=label_fontsize)
ax4.legend(fontsize=legend_fontsize)
ax4.grid(False)
ax4.annotate(f'Fitting accuracy: {test_accuracy:.2f}%', xy=(0.05, 0.85), xycoords='axes fraction',
             fontsize=annot_fontsize, color='black', bbox=dict(facecolor='white', alpha=0.6))

plt.tight_layout()
plt.show()



KeyboardInterrupt: 

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.regularizers import l2
import matplotlib.pyplot as plt
import matplotlib as mpl

# 设置新罗马字体
mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['axes.unicode_minus'] = False


# 读取数据
file_path = ''  # 请替换为你的文件路径
data = pd.read_excel(file_path)

# 转换数值类型
data.iloc[:, 1:12] = data.iloc[:, 1:12].apply(pd.to_numeric, errors='coerce')
data.dropna(inplace=True)

# 特征与目标变量
X = data.iloc[:, 1:9].values  # 第二列到第九列：测井数据
y = data.iloc[:, 9:11].values  # 第十列和第十一列：裂缝倾角、宽度

# 数据划分
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 标准化
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 构建模型函数
def build_model(optimizer):
    model = keras.Sequential([
        layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(128, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dropout(0.2),
        layers.Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
        layers.Dense(2)  # 输出裂缝倾角、宽度
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])
    return model

# 训练Adam优化器模型
adam_model = build_model(Adam(learning_rate=0.001))
history_adam = adam_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=0)

# 训练SGD优化器模型
sgd_model = build_model(SGD(learning_rate=0.001))
history_sgd = sgd_model.fit(X_train, y_train, epochs=200, batch_size=32, validation_split=0.2, verbose=0)

# === 可视化损失曲线（中文） ===
plt.figure(figsize=(12, 6), dpi=150)

# Adam优化器损失曲线
plt.subplot(1, 2, 1)
plt.plot(history_adam.history['loss'], label='Training set loss (Adam)', color='blue')
plt.plot(history_adam.history['val_loss'], label='Validation set loss (Adam)', color='orange')
plt.title('Loss profile of the Adam optimizer')
plt.xlabel('Training rounds')
plt.ylabel('Damages')
plt.legend()

# SGD优化器损失曲线
plt.subplot(1, 2, 2)
plt.plot(history_sgd.history['loss'], label='Training set loss (SGD)', color='green')
plt.plot(history_sgd.history['val_loss'], label='Validation set loss (SGD)', color='red')
plt.title('Loss profiles of the SGD optimizer')
plt.xlabel('Training rounds')
plt.ylabel('Damages')
plt.legend()

plt.tight_layout()
plt.show()

# === 模型评估 ===
loss_adam, mae_adam = adam_model.evaluate(X_test, y_test, verbose=0)
loss_sgd, mae_sgd = sgd_model.evaluate(X_test, y_test, verbose=0)

print(f"Adam优化器 - 测试集均方误差: {loss_adam:.4f}, 平均绝对误差: {mae_adam:.4f}")
print(f"SGD优化器 - 测试集均方误差: {loss_sgd:.4f}, 平均绝对误差: {mae_sgd:.4f}")

# === 计算R²值 ===
pred_adam = adam_model.predict(X_test)
pred_sgd = sgd_model.predict(X_test)

r2_adam = {
    "裂缝倾角": r2_score(y_test[:, 0], pred_adam[:, 0]),
    "裂缝宽度": r2_score(y_test[:, 1], pred_adam[:, 1])
}
r2_sgd = {
    "裂缝倾角": r2_score(y_test[:, 0], pred_sgd[:, 0]),
    "裂缝宽度": r2_score(y_test[:, 1], pred_sgd[:, 1])
}

print(f"Adam优化器 - 裂缝倾角R²: {r2_adam['裂缝倾角']:.4f}, 裂缝宽度R²: {r2_adam['裂缝宽度']:.4f}")
print(f"SGD优化器 - 裂缝倾角R²: {r2_sgd['裂缝倾角']:.4f}, 裂缝宽度R²: {r2_sgd['裂缝宽度']:.4f}")

# === 裂缝密度计算 ===
fracture_density_adam = pred_adam[:, 1] / (np.cos(np.radians(pred_adam[:, 0])) + 1e-6)
fracture_density_sgd = pred_sgd[:, 1] / (np.cos(np.radians(pred_sgd[:, 0])) + 1e-6)

# === 结果保存 ===
results_df = pd.DataFrame({
    '预测裂缝倾角（Adam）': pred_adam[:, 0],
    '预测裂缝宽度（Adam）': pred_adam[:, 1],
    '裂缝密度（Adam）': fracture_density_adam,
    '裂缝密度（SGD）': fracture_density_sgd
})

results_df.to_excel('预测结果_裂缝倾角_宽度_密度.xlsx', index=False)
print("✅ 预测结果已保存至：预测结果_裂缝倾角_宽度_密度.xlsx")


AttributeError: partially initialized module 'pandas' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)